In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/'My Drive'/MusicTransformer

In [ ]:
ls

In [ ]:
!pip install -r requirements.txt
!pip install progress
!pip install pretty_midi

In [ ]:
!pip install tensorflow==2.7.0

In [ ]:

!python preprocess.py midi_classical midi_processed


In [ ]:
import os
os.getcwd()

In [ ]:
print(os.listdir())

In [ ]:
from model import MusicTransformer
from custom.layers import *
from custom import callback
from tensorflow.python import keras
# import params as par
import midi_processor.processor as sequence
from tensorflow.python import enable_eager_execution
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
tf.executing_eagerly()

In [ ]:
tf.test.is_gpu_available()



In [ ]:
l_r = 0.001 #@param {type:"slider", min:0, max:0.1, step:0.0001}
batch_size = 1 #@param {type:"slider", min:1, max:100, step:1}
pickle_dir = "processed/" #@param {type:"string"}
max_seq = 2048 #@param {type:"slider", min:1, max:3000, step:1}
epochs = 41 #@param {type:"slider", min:1, max:10000, step:1}
model_save_path = "bin/models" #@param {type:"string"}
embedding_dim = 256 #@param {type:"slider", min:2, max:2048, step:1}

event_dim = sequence.RANGE_NOTE_ON + sequence.RANGE_NOTE_OFF + sequence.RANGE_TIME_SHIFT + sequence.RANGE_VEL
vocab_size = event_dim + 3

In [ ]:
dataset = Data('dataset/processed/')
opt = Adam(l_r)
mt = MusicTransformer(
    embedding_dim=embedding_dim, 
    vocab_size=vocab_size, 
    num_layer=3, 
    max_seq=max_seq,
    debug=False)
mt.compile(optimizer=opt, loss=callback.TransformerLoss())

# Train Model

In [ ]:
!nvidia-smi

In [ ]:
history = self.fit(prepared_data.dataset, prepared_data.dataset, epochs=epochs)
        print(history.history)

In [ ]:
    for e in range(epochs):
        for b in range(len(dataset.files) // batch_size):
            batch_x, batch_y = dataset.seq2seq_batch(batch_size, par.max_seq)
            result_metrics = mt.train_on_batch(batch_x, batch_y)
            if b % 1 == 0:
              print('===========================================\n')
              print('Loss: {:6.6}, Accuracy: {:3.2}'.format(result_metrics[0], result_metrics[1]))
              mt.save('mt-2048-h4-dim256.h5', )
              
            

In [ ]:
MusicTransformer

In [ ]:
!python train.py --epochs=1 --pickle_dir="dataset/processed" --save_path="bin/models" --max_seq=2048  --batch_size=1 --l_r=0.001

# Generate Model

In [ ]:
def fill_with_placeholder(prev_data: list, max_len: int, max_val: int=239):
    placeholder = [max_val for _ in range(max_len - len(prev_data))]
    return placeholder+prev_data

In [ ]:
for i in range(max_seq):
    

In [ ]:
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np

# suppress warning/error messages in terminal
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# load in data from mnist dataset (60k training, 10k test)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape and convert to one-hot
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# split main training set into train/validation sets (6k out of 60k data points reserved for validation)
x_validate = x_train[:6000, :]
y_validate = y_train[:6000, :]
x_train = x_train[6000:, :]
y_train = y_train[6000:, :]

'''
Building layers for the feedforward NN:
Hidden layers have 56, 96, 96, and 56 nodes, in that order.
Takes in 784 values (pixel input) and outputs 10 values (predicted probability for each number, 0-9).
'''
model = models.Sequential()
model.add(layers.Dense(56,activation='relu',input_shape=(784,)))
model.add(layers.Dense(96,activation='relu'))
model.add(layers.Dense(96,activation='relu'))
model.add(layers.Dense(56,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',
			  optimizer='adam',
			  metrics=['accuracy'])

# training the NN
epochs = 20
history = model.fit(x_train,y_train,epochs=epochs,batch_size=512,verbose=2,validation_data=(x_validate,y_validate))
print("Finished fitting.")

# plotting learning curves during training (on both training and validation data)
epoch_labels = range(1, epochs+1)
hist_dict = history.history
plt.title("Accuracy vs Epochs")
plt.plot(epoch_labels, hist_dict["acc"],'bo', label="Training")
plt.plot(epoch_labels, hist_dict["val_acc"],'go', label="Validation")
plt.legend(loc="best")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

# evaluating final performance using test data
print("Checking accuracy on test set...")
acc = model.evaluate(x_test,y_test,batch_size=512)
print("\nAccuracy on test set: " + str(acc[1]))
plt.show()